In [82]:
import pandas as pd
import math
import scipy.stats as st
from sklearn.preprocessing import MinMaxScaler


In [83]:
df=pd.read_csv("amazon_review.csv")

In [84]:
df.head(3)

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,day_diff,helpful_yes,total_vote
0,A3SBTW3WS4IQSN,B007WTAJTO,NaN,"[0, 0]",No issues.,4.0,Four Stars,1406073600,2014-07-23,138,0,0
1,A18K1ODH1I2MVB,B007WTAJTO,0mie,"[0, 0]","Purchased this for my device, it worked as adv...",5.0,MOAR SPACE!!!,1382659200,2013-10-25,409,0,0
2,A2FII3I2MBMUIA,B007WTAJTO,1K3,"[0, 0]",it works as expected. I should have sprung for...,4.0,nothing to really say....,1356220800,2012-12-23,715,0,0


In [85]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4915 entries, 0 to 4914
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   reviewerID      4915 non-null   object 
 1   asin            4915 non-null   object 
 2   reviewerName    4914 non-null   object 
 3   helpful         4915 non-null   object 
 4   reviewText      4914 non-null   object 
 5   overall         4915 non-null   float64
 6   summary         4915 non-null   object 
 7   unixReviewTime  4915 non-null   int64  
 8   reviewTime      4915 non-null   object 
 9   day_diff        4915 non-null   int64  
 10  helpful_yes     4915 non-null   int64  
 11  total_vote      4915 non-null   int64  
dtypes: float64(1), int64(4), object(7)
memory usage: 460.9+ KB


In [86]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
overall,4915.0,4.587589e+00,9.968451e-01,1.000000e+00,5.000000e+00,5.000000e+00,5.000000e+00,5.000000e+00
unixReviewTime,4915.0,1.379465e+09,1.581857e+07,1.339200e+09,1.365898e+09,1.381277e+09,1.392163e+09,1.406074e+09
day_diff,4915.0,4.373670e+02,2.094399e+02,1.000000e+00,2.810000e+02,4.310000e+02,6.010000e+02,1.064000e+03
helpful_yes,4915.0,1.311089e+00,4.161916e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.952000e+03
total_vote,4915.0,1.521465e+00,4.412309e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.020000e+03


# Görev 1: Average Rating’i güncel yorumlara göre hesaplayınız ve var olan average rating ile kıyaslayınız.

### Adım 1: Ürünün ortalama puanını hesaplayınız.

In [87]:
df["overall"].mean()

4.587589013224822

### Adım 2: Tarihe göre ağırlıklı puan ortalamasını hesaplayınız.
* reviewTime değişkenini tarih değişkeni olarak tanıtmanız
* • reviewTime'ın max değerini current_date olarak kabul etmeniz
* • her bir puan-yorum tarihi ile current_date'in farkını gün cinsinden ifade ederek yeni değişken oluşturmanız ve gün cinsinden ifade edilen değişkeni quantile fonksiyonu ile 4'e bölüp (3 çeyrek verilirse 4 parça çıkar çeyrekliklerden gelen değerlere göre ağırlıklandırma yapmanız
gerekir. Örneğin q1 = 12 ise ağırlıklandırırken 12 günden az süre önce yapılan yorumların ortalamasını alıp bunlara yüksek ağırlık vermek gibi.Ağırlıklar (35-30-20-15)

In [88]:
df["reviewTime"]=pd.to_datetime(df["reviewTime"]) 

In [89]:
current_day=df["reviewTime"].max()
current_day

Timestamp('2014-12-07 00:00:00')

In [90]:
df["comment_days"]=current_day- df["reviewTime"]

In [91]:
df.head(3)

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,day_diff,helpful_yes,total_vote,comment_days
0,A3SBTW3WS4IQSN,B007WTAJTO,NaN,"[0, 0]",No issues.,4.0,Four Stars,1406073600,2014-07-23,138,0,0,137 days
1,A18K1ODH1I2MVB,B007WTAJTO,0mie,"[0, 0]","Purchased this for my device, it worked as adv...",5.0,MOAR SPACE!!!,1382659200,2013-10-25,409,0,0,408 days
2,A2FII3I2MBMUIA,B007WTAJTO,1K3,"[0, 0]",it works as expected. I should have sprung for...,4.0,nothing to really say....,1356220800,2012-12-23,715,0,0,714 days


In [92]:
df['comment_days_quantile'] = pd.qcut(df['comment_days'], q=4, labels=['Q1', 'Q2', 'Q3', 'Q4'])

In [93]:
df.head(3)

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,day_diff,helpful_yes,total_vote,comment_days,comment_days_quantile
0,A3SBTW3WS4IQSN,B007WTAJTO,NaN,"[0, 0]",No issues.,4.0,Four Stars,1406073600,2014-07-23,138,0,0,137 days,Q1
1,A18K1ODH1I2MVB,B007WTAJTO,0mie,"[0, 0]","Purchased this for my device, it worked as adv...",5.0,MOAR SPACE!!!,1382659200,2013-10-25,409,0,0,408 days,Q2
2,A2FII3I2MBMUIA,B007WTAJTO,1K3,"[0, 0]",it works as expected. I should have sprung for...,4.0,nothing to really say....,1356220800,2012-12-23,715,0,0,714 days,Q4


In [94]:
df.loc[df["comment_days_quantile"]== "Q1"]["overall"].mean()*0.35+df.loc[df["comment_days_quantile"]== "Q2"]["overall"].mean()*0.30+df.loc[df["comment_days_quantile"]== "Q3"]["overall"].mean()*0.20+df.loc[df["comment_days_quantile"]== "Q4"]["overall"].mean()*0.15


4.615640057729592

### Adım 3: Ağırlıklandırılmış puanlamada her bir zaman diliminin ortalamasını karşılaştırıp yorumlayınız.

In [95]:
Quantiles_=['Q1', 'Q2', 'Q3', 'Q4']
for i in Quantiles_:
    print(f"{i}' nin ortalaması: ' {df.loc[df['comment_days_quantile']== i]['overall'].mean()}")

Q1' nin ortalaması: ' 4.6957928802588995
Q2' nin ortalaması: ' 4.636140637775961
Q3' nin ortalaması: ' 4.571661237785016
Q4' nin ortalaması: ' 4.4462540716612375


# Görev 2: Ürün için ürün detay sayfasında görüntülenecek 20 review’i belirleyiniz.

### Adım 1: helpful_no değişkenini üretiniz.
* • total_vote bir yoruma verilen toplam up-down sayısıdır.
* • up, helpful demektir.
* • Veri setinde helpful_no değişkeni yoktur, var olan değişkenler üzerinden üretilmesi gerekmektedir.
* • Toplam oy sayısından (total_vote) yararlı oy sayısı (helpful_yes) çıkarılarak yararlı bulunmayan oy sayılarını (helpful_no) bulunuz.

In [96]:
df["helpful_no"]=df["total_vote"]-df["helpful_yes"]

In [97]:
df.head(3)

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,day_diff,helpful_yes,total_vote,comment_days,comment_days_quantile,helpful_no
0,A3SBTW3WS4IQSN,B007WTAJTO,NaN,"[0, 0]",No issues.,4.0,Four Stars,1406073600,2014-07-23,138,0,0,137 days,Q1,0
1,A18K1ODH1I2MVB,B007WTAJTO,0mie,"[0, 0]","Purchased this for my device, it worked as adv...",5.0,MOAR SPACE!!!,1382659200,2013-10-25,409,0,0,408 days,Q2,0
2,A2FII3I2MBMUIA,B007WTAJTO,1K3,"[0, 0]",it works as expected. I should have sprung for...,4.0,nothing to really say....,1356220800,2012-12-23,715,0,0,714 days,Q4,0


In [98]:
df["helpful_no"].sum()

1034

### Adım 2: score_pos_neg_diff, score_average_rating ve wilson_lower_bound skorlarını hesaplayıp veriye ekleyiniz.
* • score_pos_neg_diff, score_average_rating ve wilson_lower_bound skorlarını hesaplayabilmek için score_pos_neg_diff,
score_average_rating ve wilson_lower_bound fonksiyonlarını tanımlayınız.
* • score_pos_neg_diff'a göre skorlar oluşturunuz. Ardından; df içerisinde score_pos_neg_diff ismiyle kaydediniz.
* • score_average_rating'a göre skorlar oluşturunuz. Ardından; df içerisinde score_average_rating ismiyle kaydediniz.
* • wilson_lower_bound'a göre skorlar oluşturunuz. Ardından; df içerisinde wilson_lower_bound ismiyle kaydediniz.

In [99]:
def score_pos_neg_diff(pos, neg):
    return pos - neg

In [100]:
df['score_pos_neg_diff'] = df.apply(lambda x: score_pos_neg_diff(x['helpful_yes'],
                                                                 x['helpful_no']), axis=1)

In [101]:
def score_average_rating(up, down):
    if up + down == 0:
        return 0
    return up / (up + down)

In [102]:
df['score_average_rating'] = df.apply(lambda x: score_average_rating(x['helpful_yes'],
                                                                     x['helpful_no']), axis=1)

In [103]:
def wilson_lower_bound(up, down, confidence=0.95):
    n = up + down
    if n == 0:
        return 0
    z = st.norm.ppf(1 - (1 - confidence) / 2)
    phat = 1.0 * up / n
    return (phat + z * z / (2 * n) - z * math.sqrt((phat * (1 - phat) + z * z / (4 * n)) / n)) / (1 + z * z / n)

In [104]:
df['wilson_lower_bound'] = df.apply(lambda x: wilson_lower_bound(x['helpful_yes'],
                                                                 x['helpful_no']), axis=1)

In [105]:
df.head(3)

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,day_diff,helpful_yes,total_vote,comment_days,comment_days_quantile,helpful_no,score_pos_neg_diff,score_average_rating,wilson_lower_bound
0,A3SBTW3WS4IQSN,B007WTAJTO,NaN,"[0, 0]",No issues.,4.0,Four Stars,1406073600,2014-07-23,138,0,0,137 days,Q1,0,0,0.0,0.0
1,A18K1ODH1I2MVB,B007WTAJTO,0mie,"[0, 0]","Purchased this for my device, it worked as adv...",5.0,MOAR SPACE!!!,1382659200,2013-10-25,409,0,0,408 days,Q2,0,0,0.0,0.0
2,A2FII3I2MBMUIA,B007WTAJTO,1K3,"[0, 0]",it works as expected. I should have sprung for...,4.0,nothing to really say....,1356220800,2012-12-23,715,0,0,714 days,Q4,0,0,0.0,0.0


In [106]:
df.sort_values("wilson_lower_bound",ascending=False).head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,day_diff,helpful_yes,total_vote,comment_days,comment_days_quantile,helpful_no,score_pos_neg_diff,score_average_rating,wilson_lower_bound
2031,A12B7ZMXFI6IXY,B007WTAJTO,"Hyoun Kim ""Faluzure""","[1952, 2020]",[[ UPDATE - 6/19/2014 ]]So my lovely wife boug...,5.0,UPDATED - Great w/ Galaxy S4 & Galaxy Tab 4 10...,1367366400,2013-01-05,702,1952,2020,701 days,Q4,68,1884,0.966337,0.957544
3449,AOEAD7DPLZE53,B007WTAJTO,NLee the Engineer,"[1428, 1505]",I have tested dozens of SDHC and micro-SDHC ca...,5.0,Top of the class among all (budget-priced) mic...,1348617600,2012-09-26,803,1428,1505,802 days,Q4,77,1351,0.948837,0.936519
4212,AVBMZZAFEKO58,B007WTAJTO,SkincareCEO,"[1568, 1694]",NOTE: please read the last update (scroll to ...,1.0,1 Star reviews - Micro SDXC card unmounts itse...,1375660800,2013-05-08,579,1568,1694,578 days,Q3,126,1442,0.925620,0.912139
317,A1ZQAQFYSXL5MQ,B007WTAJTO,"Amazon Customer ""Kelly""","[422, 495]","If your card gets hot enough to be painful, it...",1.0,"Warning, read this!",1346544000,2012-02-09,1033,422,495,1032 days,Q4,73,349,0.852525,0.818577
4672,A2DKQQIZ793AV5,B007WTAJTO,Twister,"[45, 49]",Sandisk announcement of the first 128GB micro ...,5.0,Super high capacity!!! Excellent price (on Am...,1394150400,2014-07-03,158,45,49,157 days,Q1,4,41,0.918367,0.808109
